In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)

## Imports libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output
%load_ext tensorboard

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

In [ ]:
path3 = '/home/maxwellsam/Multimodal-COVID19-Images/generatedImages/VAE/COVID-19/'
prefix3 = 'ct_covid'
path4 = "/home/maxwellsam/Multimodal-COVID19-Images/generatedImages/VAE/Non-COVID-19/"
prefix4 = 'ct_noncovid'

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
!pwd

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covid-19 negative

In [ ]:
ct_covid_features_df_gen =  processImages(path3,1)#1--> covid-19 positive
ct_noncovid_features_df_gen =  processImages(path4,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
type(cvd_imgs_dataset)

In [ ]:
cvd_imgs_gen = [ct_noncovid_features_df_gen, ct_covid_features_df_gen]
cvd_imgs_dataset_gen = pd.concat(cvd_imgs_gen)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_gen = cvd_imgs_dataset_gen.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_gen)

In [ ]:
original_images = cvd_imgs_dataset

In [ ]:
generated_images = cvd_imgs_dataset_gen

In [ ]:
type(generated_images)

#### Convert Imges to Numpy Array

In [ ]:
original_images_numpy = original_images.to_numpy()

In [ ]:
generated_images_numpy = generated_images.to_numpy()

In [ ]:
generated_images_numpy.shape

#### Convert Imges to Torch Tensors 

In [ ]:
original_images_tensors = torch.from_numpy(original_images_numpy)

In [ ]:
generated_images_tensors = torch.from_numpy(generated_images_numpy)

In [ ]:
generated_images_tensors[:4649].shape

#### Calculating the Inception Score

In [ ]:
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3
import numpy as np
from scipy.stats import entropy

def inception_score(images, splits=10, batch_size=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Ensure images are PyTorch tensors
    if isinstance(images, np.ndarray):
        images = torch.from_numpy(images)

    # Debugging: Print shape before processing
    print(f"Before Processing: {images.shape}")

    # Fix incorrectly flattened images
    if len(images.shape) == 2:  # Shape (N, Flattened)
        N, D = images.shape
        h_w = int(D ** 0.5)
        images = images.view(N, 1, h_w, h_w)  # Assume grayscale

    # Ensure images have (N, C, H, W) format
    if images.dim() != 4:
        raise ValueError(f"Images must have 4 dimensions (N, C, H, W), but got {images.shape}")

    # Convert grayscale (1 channel) to RGB
    if images.shape[1] == 1:
        images = images.repeat(1, 3, 1, 1)  # Convert (N, 1, H, W) → (N, 3, H, W)

    # Normalize images
    images = images.float()
    if images.max() > 1:
        images /= 255.0

    # Resize to 299x299 for Inception v3
    images = F.interpolate(images, size=(299, 299), mode='bilinear', align_corners=False)

    # Move to device
    images = images.to(device)
    model = inception_v3(pretrained=True, transform_input=False).eval().to(device)

    scores = []
    num_batches = (len(images) + batch_size - 1) // batch_size

    for i in range(num_batches):
        batch = images[i * batch_size: (i + 1) * batch_size]

        with torch.no_grad():
            preds = F.softmax(model(batch), dim=1).cpu().numpy()

        # Compute Inception Score for batch
        split_scores = []
        for k in range(splits):
            part = preds[k * (len(preds) // splits): (k + 1) * (len(preds) // splits), :]
            py = np.mean(part, axis=0)
            scores_batch = entropy(part.T, py[:, None], base=np.e)
            split_scores.append(np.exp(np.mean(scores_batch)))

        scores.append(np.mean(split_scores))

    return np.mean(scores), np.std(scores)


In [ ]:
generated_images_tensors.shape

In [ ]:
my_gen_images = generated_images.iloc[:,:-1].to_numpy().reshape((400,300,300,1))

In [ ]:
batch_size = 10  # Adjust this based on your GPU capacity
num_batches = (len(my_gen_images) + batch_size - 1) // batch_size  # Compute number of batches

scores = []
for i in range(num_batches):
    batch = my_gen_images[i * batch_size: (i + 1) * batch_size]
    score, _ = inception_score(batch)
    scores.append(score)

final_score = np.mean(scores)
print(f"Final Inception Score: {final_score}")


#### Structural Similarity Index

In [ ]:
from skimage.metrics import structural_similarity as ssim
import numpy as np

def calculate_ssim(img1, img2):
    ssim_value, _ = ssim(img1, img2, full=True, multichannel=True)
    return ssim_value


In [ ]:
calculate_ssim(original_images_numpy[:400], generated_images_numpy[:400])

#### Fréchet Inception Distance

In [ ]:
from torchvision.models.inception import inception_v3
from torch.nn.functional import adaptive_avg_pool2d
import numpy as np
from scipy.linalg import sqrtm

def calculate_fid(real_images, fake_images):
    model = inception_v3(pretrained=True, transform_input=False).eval().cuda()
    real_features = get_features(model, real_images)
    fake_features = get_features(model, fake_images)

    mu_real, sigma_real = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu_fake, sigma_fake = fake_features.mean(axis=0), np.cov(fake_features, rowvar=False)
    ssdiff = np.sum((mu_real - mu_fake) ** 2)
    covmean = sqrtm(sigma_real.dot(sigma_fake))

    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma_real + sigma_fake - 2 * covmean)
    return fid

def get_features(model, images):
    features = []
    with torch.no_grad():
        for img in images:
            img = img.unsqueeze(0).cuda()
            feature = model(img).squeeze().cpu().numpy()
            features.append(feature)
    return np.array(features)


In [ ]:
original_image_FID = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))

In [ ]:
original_image_tensor_FID = torch.from_numpy(original_image_FID)

In [ ]:
import torch.nn.functional as F

# Assume my_gen_tensor__ has shape [14012, 300, 300, 1]
# Permute to shape [14012, 1, 300, 300]
images_ = original_image_tensor_FID.permute(0, 3, 1, 2)

# Now pass the images through the interpolation function
images_resized_ = F.interpolate(images_, size=(299, 299), mode='bilinear', align_corners=False)

# Check the new shape
print("Resized images shape:", images_resized_.shape)


In [ ]:
import torch


def convert_to_rgb(images):
    return images.repeat(1, 3, 1, 1)

In [ ]:
real_images_rgb = convert_to_rgb(original_image_tensor_FID[:10])
fake_images_rgb = convert_to_rgb(images_resized[:10])

In [ ]:
type(fake_images_rgb.float().cuda())

In [ ]:
calculate_fid(real_images_rgb.float().cuda(), fake_images_rgb.float().cuda())

#### Learned Perceptual Image Patch Similarity

In [ ]:
pip install lpips

In [ ]:
import lpips
import torch

def calculate_lpips(img1, img2):
    loss_fn = lpips.LPIPS(net='vgg')  

    img1 = img1[:, :-1].view(-1, 1, 300, 300) 
    img2 = img2[:, :-1].view(-1, 1, 300, 300)
    img1 = img1.repeat(1, 3, 1, 1)  
    img2 = img2.repeat(1, 3, 1, 1)

    lpips_value = loss_fn(img1, img2)
    return lpips_value.mean().item() 

In [ ]:
calculate_lpips(original_images_tensors[:100], generated_images_tensors[:100])